# Network Deliberation ABM
## Discrete state, generated NK Model task

In [ ]:
# Imports
import configparser
import math
import random

import networkx as nx
import nkmodel as nk
import numpy as np
from numpy import random as nprand
import repsci
from tqdm.notebook import tqdm

import netdelib.soclearn as slearn
import netdelib.soclearn.evaluate as sleval
import netdelib.soclearn.models.generated as slgen
import netdelib.soclearn.strategy as slstrat
from netdelib.topologies.factories import *

from discrete import run_discrete_trial

In [ ]:
# Configuration
N = 100
M = 5
runs = 100
stages = 50
steps = 1
bit_count = 15
K = 7
nk_exponent = 8

# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (20, 20),
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False})

# Configure Networks
networks = {
    'Complete': CompleteFactory(N, M),
    'Lattice': SmallWorldFactory(N, M, 9, 0)
}

# Configure Strategies
individual = True
individual_mode = slearn.MODE_ALL
sample = 3
learning_strategies = {
    'Best Neighbor': slstrat.best_neighbor,
    'Conform': slstrat.conform
}

# Run Simulation 

In [ ]:
experiment = repsci.Experiment("nk_barkoczi_galesic")
log = experiment.get_logger()

num_net = len(networks)
num_strat = len(learning_strategies)
score = [[0 for net in range(num_net)] for strat in range(num_strat)]
distance_score = [[0 for net in range(num_net)] for strat in range(num_strat)]
correct = [[0 for net in range(num_net)] for strat in range(num_strat)]
score_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
distance_score_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
correct_for_strat_net_run = [[[] for net in range(num_net)] for strat in range(num_strat)]
max_values = []

log.info('Running trials')
for run in tqdm(range(runs)):
    log.info('  Trial {}/{}'.format(run, runs))
    model = nk.NK(bit_count, K, nk_exponent)
    objective = model.get_value
    log.info('    Finding global max')
    max_state, max_value = model.get_global_max()
    max_values.append(max_state)
    log.info('    Generating initial beliefs')
    initial_beliefs = slgen.initial_beliefs_noisy(N, max_state, p_error=0.5)
    for i, (title, learning_strategy) in enumerate(learning_strategies.items()):
        log.info('    Strategy: {}'.format(title))
        for j, (title2, factory) in enumerate(networks.items()):
            log.info('      Network: {}'.format(title2))
            # Record time series for scores
            run_score = []
            beliefs_at_step = run_discrete_trial(
                factory,
                learning_strategy,
                initial_beliefs,
                objective,
                N, M, stages, steps,
                individual=individual,
                individual_mode=individual_mode,
                sample=sample)
            scores_for_step_agent = [model.get_values(beliefs) for beliefs in beliefs_at_step]
            
            # NK Model score
            # Have to convert scores from dict values to list, otherwise np gets confused
            score_for_step = [
                np.divide(np.mean(list(scores.values())), max_value)
                for scores in scores_for_step_agent]
            score_for_strat_net_run[i][j].append(score_for_step)
            # Fraction correct
            correct_at_step = [
                sum([1 for belief in beliefs.values() if tuple(belief) == tuple(max_state)]) / N
                for beliefs in beliefs_at_step]
            correct_for_strat_net_run[i][j].append(correct_at_step)
            # Average distance to correct solution
            distance_score_for_step = sleval.belief_distance(beliefs_at_step, max_state)
            distance_score_for_strat_net_run[i][j].append(distance_score_for_step)
log.info('All trials complete')

# Mean/std score for all runs
log.info('Calculating trial statistics')
for i, (title, learning_strategy) in enumerate(learning_strategies.items()):
    for j, (title2, factory) in enumerate(networks.items()):
        # Average NK model score
        mean = np.mean(score_for_strat_net_run[i][j], axis=0)
        std = np.std(score_for_strat_net_run[i][j], axis=0)
        score[i][j] = (mean, std)
        # Fraction correct
        mean = np.mean(correct_for_strat_net_run[i][j], axis=0)
        std = np.std(correct_for_strat_net_run[i][j], axis=0)
        correct[i][j] = (mean, std)
        # Distance score
        mean = np.mean(distance_score_for_strat_net_run[i][j], axis=0)
        std = np.std(distance_score_for_strat_net_run[i][j], axis=0)
        distance_score[i][j] = (mean, std)

In [ ]:
out = experiment.get_filename('score.npy')
np.save(out, score)
out = experiment.get_filename('correct.npy')
np.save(out, correct)
out = experiment.get_filename('distance_score.npy')
np.save(out, distance_score)

In [ ]:
fmt = ['r-', 'b-']
plt.figure(figsize=(15, 9))
for i, (title, learning_strategy) in enumerate(learning_strategies.items()):
    plt.subplot(num_strat, num_net, 1 + i)

    for j, (network_title, network) in enumerate(networks.items()):
        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, fmt[j], label=network_title)

    plt.title(title)
    plt.ylabel('NK Value')
    plt.xlabel('Time Step')
    plt.grid(True)
    plt.tight_layout()
    plt.ylim([0.0, 1])
    plt.legend()

out = experiment.get_filename('plot.png')
plt.savefig(out, dpi=300)